In [15]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [17]:
tf.keras.backend.clear_session()

In [18]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [19]:
subject = 'Tocantins - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [20]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_TO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Tocantins - Produção de Cimento (t)', axis=1)
data

,Ano,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - IDH Renda,Tocantins - IDH Longevidade,Tocantins - Desemprego,Tocantins - IDH,Tocantins - IDH Educacao,...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Tocantins - Consumo de Cimento (t)
0,2008,1.715433e+07,1.552702e+06,10.598411,2.786493e+07,0.673182,0.783419,8.013079,0.718281,0.700364,...,2.868108,11.760983,36.939311,978.592488,1332.831312,27.105824,18.233645,62.880792,1.689588,450.745000
1,2009,1.731978e+07,1.569541e+06,10.617459,2.826546e+07,0.674813,0.785421,8.041614,0.720575,0.704145,...,2.305885,9.472585,36.787648,978.478936,1329.761783,27.155678,18.214031,62.735136,1.698582,467.588000
2,2010,1.782790e+07,1.443676e+06,10.617472,2.605538e+07,0.676007,0.786976,8.006059,0.722443,0.707452,...,1.958246,9.332485,34.377788,978.429797,1327.746981,27.093093,17.937541,62.248835,1.689662,487.208000
3,2011,1.829602e+07,1.330737e+06,10.632146,2.434482e+07,0.676774,0.788093,7.993875,0.723894,0.710276,...,2.079088,10.985689,31.445708,978.566107,1339.288604,27.200519,17.906596,61.759514,1.630499,458.331000
4,2012,1.873679e+07,1.258451e+06,10.650202,2.323840e+07,0.676869,0.788709,9.385999,0.724531,0.711858,...,2.183282,8.159014,31.919550,978.672066,1341.129295,27.263832,18.110772,62.123850,1.542457,508.683000
5,2013,1.913329e+07,1.228870e+06,10.673643,2.277987e+07,0.677108,0.789018,9.822894,0.725742,0.714846,...,2.403445,7.913561,34.865387,978.830134,1337.754475,27.211674,18.470708,63.514140,1.474065,525.234000
6,2014,1.945461e+07,1.234495e+06,10.702690,2.301322e+07,0.676867,0.788936,10.192991,0.726304,0.716833,...,2.748397,10.350442,45.725826,978.988335,1363.511779,27.574024,18.568157,62.927666,1.510968,426.767286
7,2015,1.971544e+07,1.239338e+06,10.728749,2.324825e+07,0.675606,0.788430,10.501090,0.725413,0.715819,...,2.501529,12.454398,48.162288,978.983844,1381.825545,27.799714,18.471946,62.070670,1.531417,454.929429
8,2016,1.989200e+07,1.246862e+06,10.746325,2.347786e+07,0.683859,0.797827,10.392740,0.735726,0.729669,...,2.073464,13.111908,40.542577,978.957795,1390.699392,27.680489,18.345425,62.095581,1.511336,489.016286
9,2017,1.998196e+07,1.262231e+06,10.764439,2.371682e+07,0.684301,0.797174,9.894472,0.735883,0.730275,...,2.177386,9.479812,37.984321,979.036017,1384.229204,27.536429,18.412455,62.734632,1.476663,500.270000


In [21]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - IDH Renda,Tocantins - IDH Longevidade,Tocantins - Desemprego,Tocantins - IDH,Tocantins - IDH Educacao,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,1.715433e+07,1.552702e+06,10.598411,2.786493e+07,0.673182,0.783419,8.013079,0.718281,0.700364,5.483335e+08,...,-7723.574973,2.868108,11.760983,36.939311,978.592488,1332.831312,27.105824,18.233645,62.880792,1.689588
1,1.731978e+07,1.569541e+06,10.617459,2.826546e+07,0.674813,0.785421,8.041614,0.720575,0.704145,5.302764e+08,...,-8961.781709,2.305885,9.472585,36.787648,978.478936,1329.761783,27.155678,18.214031,62.735136,1.698582
2,1.782790e+07,1.443676e+06,10.617472,2.605538e+07,0.676007,0.786976,8.006059,0.722443,0.707452,5.046425e+08,...,-9895.682385,1.958246,9.332485,34.377788,978.429797,1327.746981,27.093093,17.937541,62.248835,1.689662
3,1.829602e+07,1.330737e+06,10.632146,2.434482e+07,0.676774,0.788093,7.993875,0.723894,0.710276,4.854622e+08,...,-9989.170714,2.079088,10.985689,31.445708,978.566107,1339.288604,27.200519,17.906596,61.759514,1.630499
4,1.873679e+07,1.258451e+06,10.650202,2.323840e+07,0.676869,0.788709,9.385999,0.724531,0.711858,4.867661e+08,...,-9429.326328,2.183282,8.159014,31.919550,978.672066,1341.129295,27.263832,18.110772,62.123850,1.542457
5,1.913329e+07,1.228870e+06,10.673643,2.277987e+07,0.677108,0.789018,9.822894,0.725742,0.714846,5.225848e+08,...,-7157.441850,2.403445,7.913561,34.865387,978.830134,1337.754475,27.211674,18.470708,63.514140,1.474065
6,1.945461e+07,1.234495e+06,10.702690,2.301322e+07,0.676867,0.788936,10.192991,0.726304,0.716833,5.676413e+08,...,-3453.461327,2.748397,10.350442,45.725826,978.988335,1363.511779,27.574024,18.568157,62.927666,1.510968
7,1.971544e+07,1.239338e+06,10.728749,2.324825e+07,0.675606,0.788430,10.501090,0.725413,0.715819,5.179392e+08,...,-1442.919387,2.501529,12.454398,48.162288,978.983844,1381.825545,27.799714,18.471946,62.070670,1.531417
8,1.989200e+07,1.246862e+06,10.746325,2.347786e+07,0.683859,0.797827,10.392740,0.735726,0.729669,4.524144e+08,...,-3128.377228,2.073464,13.111908,40.542577,978.957795,1390.699392,27.680489,18.345425,62.095581,1.511336
9,1.998196e+07,1.262231e+06,10.764439,2.371682e+07,0.684301,0.797174,9.894472,0.735883,0.730275,4.213689e+08,...,-5386.337108,2.177386,9.479812,37.984321,979.036017,1384.229204,27.536429,18.412455,62.734632,1.476663


In [22]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     467.588000
1     487.208000
2     458.331000
3     508.683000
4     525.234000
5     426.767286
6     454.929429
7     489.016286
8     500.270000
9     396.438000
10    402.932000
11    500.126000
12    542.030000
13    554.321000
14           NaN
Name: Tocantins - Consumo de Cimento (t), dtype: float64

In [23]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - IDH Renda,Tocantins - IDH Longevidade,Tocantins - Desemprego,Tocantins - IDH,Tocantins - IDH Educacao,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.767496,1.899747,-1.459873,1.876486,-1.384560,-1.520880,-1.346093,-1.507908,-1.588970,1.204752,...,-0.569731,2.026352,0.984445,-0.089637,-0.913469,-1.038127,-1.221067,-0.397897,0.543546,1.392168
1,-1.604321,2.041562,-1.150826,2.100561,-0.961462,-1.083547,-1.316292,-1.127752,-1.222950,0.820657,...,-0.838980,0.085558,-0.051710,-0.120394,-1.403474,-1.167850,-1.013699,-0.478557,0.262270,1.479942
2,-1.103172,0.981551,-1.150609,0.864160,-0.651622,-0.743889,-1.353425,-0.818255,-0.902870,0.275392,...,-1.042057,-1.114490,-0.115145,-0.609104,-1.615522,-1.252999,-1.274019,-1.615599,-0.676828,1.392885
3,-0.641474,0.030395,-0.912541,-0.092787,-0.452857,-0.499842,-1.366149,-0.577975,-0.629550,-0.132596,...,-1.062386,-0.697344,0.633403,-1.203717,-1.027311,-0.765233,-0.827186,-1.742858,-1.621758,0.815509
4,-0.206749,-0.578393,-0.619594,-0.711759,-0.428007,-0.365337,0.087752,-0.472336,-0.476387,-0.104860,...,-0.940648,-0.337667,-0.646476,-1.107624,-0.570070,-0.687443,-0.563837,-0.903202,-0.918188,-0.043692
5,0.184315,-0.827521,-0.239270,-0.968273,-0.365980,-0.297827,0.544035,-0.271705,-0.187147,0.657046,...,-0.446625,0.422337,-0.757614,-0.510220,0.112033,-0.830067,-0.780786,0.577007,1.766607,-0.711136
6,0.501220,-0.780142,0.232004,-0.837728,-0.428489,-0.315710,0.930556,-0.178617,0.005202,1.615453,...,0.358807,1.613108,0.345772,1.692233,0.794710,0.258473,0.726400,0.977757,0.634065,-0.350996
7,0.758474,-0.739358,0.654789,-0.706246,-0.755615,-0.426099,1.252327,-0.326296,-0.092938,0.558230,...,0.796000,0.760923,1.298413,2.186338,0.775330,1.032439,1.665147,0.582100,-1.020884,-0.151437
8,0.932609,-0.675994,0.939945,-0.577792,1.385239,1.626408,1.139169,1.382513,1.247619,-0.835563,...,0.429496,-0.716758,1.596124,0.641091,0.662921,1.407460,1.169234,0.061789,-0.972778,-0.347402
9,1.021336,-0.546554,1.233842,-0.444113,1.499833,1.483956,0.618790,1.408515,1.306302,-1.495939,...,-0.061498,-0.358019,-0.048438,0.122287,1.000468,1.134021,0.570022,0.337445,0.261296,-0.685784


In [24]:
reshaped_train_input = dfToInputRNN(train_input)

In [25]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.88742131, -0.35267827,  1.17766484, -0.1863501 ,
          1.05319   ,  0.88979364,  0.33727699,  1.08591156,
          1.13944838, -1.0453709 ,  1.21859064, -0.99976018,
          0.79428371,  1.15575749, -0.52559858,  1.30260559,
         -0.55253046, -0.25130359,  1.38787176,  1.47154451,
         -1.80649616,  1.09296291,  0.94281138,  0.52836084,
          0.61258103,  0.87656257,  1.47801536,  1.71982086,
          1.28221181,  1.39086472,  1.58209844,  1.08872091,
          0.55212962, -1.81398118,  1.44113674,  1.11191094,
          1.30470835,  0.86390516, -1.67365968, -1.70738939,
         -1.55813196, -0.48498301, -0.75714639,  0.40770893,
         -0.27394734,  2.02323219,  1.2908951 ,  1.70465513,
          1.81148084,  1.39968653, -1.36191442,  1.28024593,
          1.11054795,  0.12662799,  1.40414112,  1.40468999,
          1.45643959,  1.58509336,  1.32283157,  1.30222047,
          1.55138602,  1.91801602, -0.77489784,  2.32098884,
         -1.1942541 , -1

In [26]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     467.588000
1     487.208000
2     458.331000
3     508.683000
4     525.234000
5     426.767286
6     454.929429
7     489.016286
8     500.270000
9     396.438000
10    402.932000
11    500.126000
Name: Tocantins - Consumo de Cimento (t), dtype: float64

In [27]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    500.126
Name: Tocantins - Consumo de Cimento (t), dtype: float64

In [28]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - IDH Renda,Tocantins - IDH Longevidade,Tocantins - Desemprego,Tocantins - IDH,Tocantins - IDH Educacao,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.617482,-0.223475,0.62504,-0.063815,0.809469,0.406857,0.06974,0.831493,0.98514,1.108405,...,0.964765,-1.482779,-2.260447,-1.030773,0.247713,0.771849,0.844429,0.498536,-1.020842,-2.164292


In [29]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.61748247, -0.22347525,  0.62504018, -0.06381543,
          0.80946923,  0.40685697,  0.06974029,  0.83149267,
          0.98514047,  1.10840491,  2.63582309, -0.18995911,
          0.72640489,  2.66833308, -0.032632  ,  0.98802435,
          0.24980494, -0.08955808,  1.85434394,  2.02706568,
         -2.21150703,  1.42938591,  0.85132668,  0.14092386,
          0.30902324,  1.00267904,  2.27970137,  1.11272593,
          0.61007902,  0.89064538,  1.42231438,  1.02650562,
          1.47127593, -1.53201473,  1.73503858,  1.66610613,
          1.0456894 ,  0.55289259, -1.81216593, -2.10342296,
         -2.55377478,  0.41940628, -1.75756445,  0.241023  ,
         -0.98659345,  1.52184916,  1.01398607,  1.49290967,
          1.40764092,  1.87930411, -2.51892072,  2.83356214,
          1.58620924, -0.02524128,  1.54231542,  1.5176119 ,
          1.7355269 ,  1.87941982,  1.54198129,  1.51276712,
          1.83795594,  2.62210693,  1.04993174,  0.96476493,
         -1.48277861, -2

In [30]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    542.03
Name: Tocantins - Consumo de Cimento (t), dtype: float64

In [31]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [32]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [33]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3907076873, 859915381, 1530994990, 2344165348, 4290868631, 4175931867, 291033947, 320012396, 1028761098, 3125942825, 380213353, 1911872039, 1999809643, 1726877348, 3238861946, 1121860329, 2161500313, 2855792509, 2824497702, 3162195277, 4192068278, 2957388035, 698066471, 3895513248, 1400639704, 1910399400, 3332770792, 4263308427, 1917147453, 961438002, 3333597647, 2614490837, 2693529714, 4266071995, 2035320070, 2961365688, 726488200, 2816025948, 1695274190, 2589617050, 1319032797, 1503741223, 2186034190, 1331056956, 611744635, 3105546823, 4111615436, 3196406452, 3449906687, 93977976, 128903034, 3461580259, 4076148294, 3158913831, 1898077413, 3374921039, 2369016292, 3404789623, 1596025423, 2167083874, 822855587, 2511140606, 3890255289, 1593379346, 2419289444, 3762243648, 703488675, 2965108525, 44224195, 144992594, 1960797297, 1289503414, 1115710354, 1332092050, 2661290448, 602762207, 2938556764, 4161844678, 518585116, 3646134594, 734342683, 2132907297, 1823642570, 710665562, 4261231023,

2023-09-13 16:01:57.489527: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-13 16:01:57.507515: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-13 16:01:57.507709: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

loss: 38.998897552490234
winner_seed: 3907076873


Step: 1 ___________________________________________
loss: 34.708457946777344
winner_seed: 859915381


Step: 2 ___________________________________________
loss: 45.76688766479492


Step: 3 ___________________________________________
loss: 51.311214447021484


Step: 4 ___________________________________________
loss: 43.950164794921875


Step: 5 ___________________________________________
loss: 49.3082389831543


Step: 6 ___________________________________________
loss: 29.4029541015625
winner_seed: 291033947


Step: 7 ___________________________________________
loss: 66.839111328125


Step: 8 ___________________________________________
loss: 58.487091064453125


Step: 9 ___________________________________________
loss: 39.89474105834961


Step: 10 ___________________________________________
loss: 44.80112075805664


Step: 11 ___________________________________________
loss: 30.43915367126465


Step: 12 __________________________________

In [34]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 220731.2500
Epoch 2/10000
1/1 [==============================] - 0s 8ms/step - loss: 215671.2969
Epoch 3/10000
1/1 [==============================] - 0s 8ms/step - loss: 206889.6406
Epoch 4/10000
1/1 [==============================] - 0s 8ms/step - loss: 166997.2812
Epoch 5/10000
1/1 [==============================] - 0s 8ms/step - loss: 22579.7266
Epoch 6/10000
1/1 [==============================] - 0s 7ms/step - loss: 7553.0454
Epoch 7/10000
1/1 [==============================] - 0s 8ms/step - loss: 5281.1719
Epoch 8/10000
1/1 [==============================] - 0s 7ms/step - loss: 2896.9421
Epoch 9/10000
1/1 [==============================] - 0s 8ms/step - loss: 1724.2500
Epoch 10/10000
1/1 [==============================] - 0s 7ms/step - loss: 1639.9103
Epoch 11/10000
1/1 [==============================] - 0s 8ms/step - loss: 1038.6782
Epoch 12/10000
1/1 [==============================] - 0s 4ms/step - loss: 156

1/1 [==============================] - 0s 4ms/step - loss: 568.4697
Epoch 100/10000
1/1 [==============================] - 0s 6ms/step - loss: 197.4952
Epoch 101/10000
1/1 [==============================] - 0s 5ms/step - loss: 169.3945
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 287.8574
Epoch 103/10000
1/1 [==============================] - 0s 5ms/step - loss: 418.6704
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 426.7949
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 369.1634
Epoch 106/10000
1/1 [==============================] - 0s 6ms/step - loss: 259.3878
Epoch 107/10000
1/1 [==============================] - 0s 5ms/step - loss: 193.9170
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - loss: 428.7508
Epoch 109/10000
1/1 [==============================] - 0s 6ms/step - loss: 366.0623
Epoch 110/10000
1/1 [==============================] - 0s 5ms/step - loss: 463.3661
Epoch 11

1/1 [==============================] - 0s 5ms/step - loss: 413.9093
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 361.9894
Epoch 198/10000
1/1 [==============================] - 0s 8ms/step - loss: 109.1304
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 190.6451
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 352.1244
Epoch 201/10000
1/1 [==============================] - 0s 5ms/step - loss: 123.6664
Epoch 202/10000
1/1 [==============================] - 0s 5ms/step - loss: 326.2766
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - loss: 368.4966
Epoch 204/10000
1/1 [==============================] - 0s 6ms/step - loss: 209.2082
Epoch 205/10000
1/1 [==============================] - 0s 5ms/step - loss: 454.1140
Epoch 206/10000
1/1 [==============================] - 0s 5ms/step - loss: 299.1844
Epoch 207/10000
1/1 [==============================] - 0s 5ms/step - loss: 456.9544
Epoch 20

1/1 [==============================] - 0s 5ms/step - loss: 275.5140
Epoch 294/10000
1/1 [==============================] - 0s 5ms/step - loss: 425.6798
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 428.9139
Epoch 296/10000
1/1 [==============================] - 0s 6ms/step - loss: 247.2793
Epoch 297/10000
1/1 [==============================] - 0s 5ms/step - loss: 162.3123
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss: 238.4376
Epoch 299/10000
1/1 [==============================] - 0s 5ms/step - loss: 464.4067
Epoch 300/10000
1/1 [==============================] - 0s 5ms/step - loss: 323.9165
Epoch 301/10000
1/1 [==============================] - 0s 4ms/step - loss: 486.2787
Epoch 302/10000
1/1 [==============================] - 0s 5ms/step - loss: 252.2325
Epoch 303/10000
1/1 [==============================] - 0s 4ms/step - loss: 348.8501
Epoch 304/10000
1/1 [==============================] - 0s 4ms/step - loss: 887.8977
Epoch 30

1/1 [==============================] - 0s 4ms/step - loss: 406.8547
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 273.8591
Epoch 392/10000
1/1 [==============================] - 0s 6ms/step - loss: 691.8833
Epoch 393/10000
1/1 [==============================] - 0s 5ms/step - loss: 245.4140
Epoch 394/10000
1/1 [==============================] - 0s 4ms/step - loss: 341.4571
Epoch 395/10000
1/1 [==============================] - 0s 5ms/step - loss: 480.2622
Epoch 396/10000
1/1 [==============================] - 0s 4ms/step - loss: 432.5389
Epoch 397/10000
1/1 [==============================] - 0s 4ms/step - loss: 504.5067
Epoch 398/10000
1/1 [==============================] - 0s 5ms/step - loss: 217.8215
Epoch 399/10000
1/1 [==============================] - 0s 4ms/step - loss: 189.0278
Epoch 400/10000
1/1 [==============================] - 0s 4ms/step - loss: 314.3299
Epoch 401/10000
1/1 [==============================] - 0s 5ms/step - loss: 191.3920
Epoch 40

...

1/1 [==============================] - 0s 4ms/step - loss: 387.4369
Epoch 876/10000
1/1 [==============================] - 0s 4ms/step - loss: 240.8694
Epoch 877/10000
1/1 [==============================] - 0s 5ms/step - loss: 193.6564
Epoch 878/10000
1/1 [==============================] - 0s 4ms/step - loss: 231.6861
Epoch 879/10000
1/1 [==============================] - 0s 4ms/step - loss: 222.4985
Epoch 880/10000
1/1 [==============================] - 0s 5ms/step - loss: 163.0656
Epoch 881/10000
1/1 [==============================] - 0s 5ms/step - loss: 287.7360
Epoch 882/10000
1/1 [==============================] - 0s 4ms/step - loss: 270.8458
Epoch 883/10000
1/1 [==============================] - 0s 5ms/step - loss: 235.5148
Epoch 884/10000
1/1 [==============================] - 0s 4ms/step - loss: 98.0036
Epoch 885/10000
1/1 [==============================] - 0s 4ms/step - loss: 227.4077
Epoch 886/10000
1/1 [==============================] - 0s 4ms/step - loss: 174.1455
Epoch 887

1/1 [==============================] - 0s 4ms/step - loss: 257.1526
Epoch 973/10000
1/1 [==============================] - 0s 6ms/step - loss: 311.5390
Epoch 974/10000
1/1 [==============================] - 0s 5ms/step - loss: 303.3716
Epoch 975/10000
1/1 [==============================] - 0s 5ms/step - loss: 144.3036
Epoch 976/10000
1/1 [==============================] - 0s 5ms/step - loss: 383.2235
Epoch 977/10000
1/1 [==============================] - 0s 5ms/step - loss: 243.1497
Epoch 978/10000
1/1 [==============================] - 0s 4ms/step - loss: 179.8752
Epoch 979/10000
1/1 [==============================] - 0s 4ms/step - loss: 262.8648
Epoch 980/10000
1/1 [==============================] - 0s 4ms/step - loss: 206.8876
Epoch 981/10000
1/1 [==============================] - 0s 4ms/step - loss: 227.2193
Epoch 982/10000
1/1 [==============================] - 0s 5ms/step - loss: 106.9365
Epoch 983/10000
1/1 [==============================] - 0s 5ms/step - loss: 378.8783
Epoch 98

1/1 [==============================] - 0s 4ms/step - loss: 547.4648
Epoch 1069/10000
1/1 [==============================] - 0s 4ms/step - loss: 447.8313
Epoch 1070/10000
1/1 [==============================] - 0s 5ms/step - loss: 250.9501
Epoch 1071/10000
1/1 [==============================] - 0s 4ms/step - loss: 288.6344
Epoch 1072/10000
1/1 [==============================] - 0s 4ms/step - loss: 149.0015
Epoch 1073/10000
1/1 [==============================] - 0s 6ms/step - loss: 201.2853
Epoch 1074/10000
1/1 [==============================] - 0s 4ms/step - loss: 163.9026
Epoch 1075/10000
1/1 [==============================] - 0s 4ms/step - loss: 207.0201
Epoch 1076/10000
1/1 [==============================] - 0s 5ms/step - loss: 238.8000
Epoch 1077/10000
1/1 [==============================] - 0s 4ms/step - loss: 262.3645
Epoch 1078/10000
1/1 [==============================] - 0s 4ms/step - loss: 284.1482
Epoch 1079/10000
1/1 [==============================] - 0s 4ms/step - loss: 414.97

1/1 [==============================] - 0s 4ms/step - loss: 81.9093
Epoch 1165/10000
1/1 [==============================] - 0s 7ms/step - loss: 202.1673
Epoch 1166/10000
1/1 [==============================] - 0s 5ms/step - loss: 322.3417
Epoch 1167/10000
1/1 [==============================] - 0s 4ms/step - loss: 299.6750
Epoch 1168/10000
1/1 [==============================] - 0s 5ms/step - loss: 144.3944
Epoch 1169/10000
1/1 [==============================] - 0s 5ms/step - loss: 358.8991
Epoch 1170/10000
1/1 [==============================] - 0s 5ms/step - loss: 559.9204
Epoch 1171/10000
1/1 [==============================] - 0s 5ms/step - loss: 403.3637
Epoch 1172/10000
1/1 [==============================] - 0s 4ms/step - loss: 136.9580
Epoch 1173/10000
1/1 [==============================] - 0s 4ms/step - loss: 296.9613
Epoch 1174/10000
1/1 [==============================] - 0s 5ms/step - loss: 287.0688
Epoch 1175/10000
1/1 [==============================] - 0s 4ms/step - loss: 116.196

1/1 [==============================] - 0s 5ms/step - loss: 314.5334
Epoch 1261/10000
1/1 [==============================] - 0s 4ms/step - loss: 196.5473
Epoch 1262/10000
1/1 [==============================] - 0s 4ms/step - loss: 193.5892
Epoch 1263/10000
1/1 [==============================] - 0s 5ms/step - loss: 183.5635
Epoch 1264/10000
1/1 [==============================] - 0s 5ms/step - loss: 244.6544
Epoch 1265/10000
1/1 [==============================] - 0s 6ms/step - loss: 251.6465
Epoch 1266/10000
1/1 [==============================] - 0s 5ms/step - loss: 195.6014
Epoch 1267/10000
1/1 [==============================] - 0s 5ms/step - loss: 480.7407
Epoch 1268/10000
1/1 [==============================] - 0s 4ms/step - loss: 203.5758
Epoch 1269/10000
1/1 [==============================] - 0s 5ms/step - loss: 239.8795
Epoch 1270/10000
1/1 [==============================] - 0s 4ms/step - loss: 203.6934
Epoch 1271/10000
1/1 [==============================] - 0s 5ms/step - loss: 214.34

In [35]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 611ms/step
[542.03](test_target) - [[505.5437]](prediction) = 36.48629882812497


In [36]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.06731416864034274

In [37]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [38]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - IDH Renda,Tocantins - IDH Longevidade,Tocantins - Desemprego,Tocantins - IDH,Tocantins - IDH Educacao,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.975826,0.550803,-1.414213,0.488506,-1.282735,-1.272718,-0.466423,-1.264362,-1.251147,1.148205,...,1.277766,1.308949,1.412346,0.770747,1.351519,1.300143,-0.458201,0.778403,0.959304,-0.715791
1,-0.398553,0.852633,0.706362,0.905104,0.125670,0.102372,1.389428,0.083509,0.054632,0.140893,...,-0.114016,-0.190804,-0.643250,0.641496,-0.315139,-0.168177,1.387780,0.633328,0.420238,1.414178
2,1.374379,-1.403436,0.707852,-1.393610,1.157065,1.170346,-0.923006,1.180853,1.196515,-1.289098,...,-1.163750,-1.118144,-0.769096,-1.412243,-1.036380,-1.131965,-0.929579,-1.411731,-1.379542,-0.698387


train_target:


,Tocantins - Consumo de Cimento (t)
0,467.588
1,487.208
2,458.331


test_input:


...

,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - IDH Renda,Tocantins - IDH Longevidade,Tocantins - Desemprego,Tocantins - IDH,Tocantins - IDH Educacao,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.617482,-0.223475,0.62504,-0.063815,0.809469,0.406857,0.06974,0.831493,0.98514,1.108405,...,0.964765,-1.482779,-2.260447,-1.030773,0.247713,0.771849,0.844429,0.498536,-1.020842,-2.164292


test_target:


,Tocantins - Consumo de Cimento (t)
12,542.03


1/1 [==============================] - 1s 613ms/step
Error: 36.48629882812497




[450.2460021972656,
 513.4358520507812,
 455.5587158203125,
 420.25689697265625,
 445.4308166503906,
 479.1761474609375,
 496.5440979003906,
 391.0702209472656,
 405.09295654296875,
 505.543701171875]

In [39]:
display(targets)
display(predictions)

[508.683,
 525.234,
 426.7672857142857,
 454.92942857142856,
 489.0162857142857,
 500.27,
 396.438,
 402.932,
 500.126,
 542.03]

[450.2460021972656,
 513.4358520507812,
 455.5587158203125,
 420.25689697265625,
 445.4308166503906,
 479.1761474609375,
 496.5440979003906,
 391.0702209472656,
 405.09295654296875,
 505.543701171875]

In [40]:
mae = mean_absolute_error(predictions, targets)
mae

44.1865648297991

In [41]:
porcentage = mae/np.mean(targets)
porcentage

0.09309439319142257